In [ ]:
"""
setup_vqe_environment.py

This module imports and configures all dependencies needed to:
- Load and run saved HDF5 circuit data,
- Build and transpile Qiskit circuits (ansatz, unitary gates, etc.),
- Set up IBM Quantum Runtime and Aer simulators,
- Define Hamiltonians and Pauli operators,
- Run VQE simulation with SciPy optimizers,
- Perform plotting and data analysis (matplotlib, pandas),
- Leverage SciPy for classical minimization,
- Handle file I/O (h5py, os, glob, re),
- Utilize concurrency for parallel tasks.
"""

# ————————————————————————————————————————————————
# Standard library
import os
import glob
import re
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed

# ————————————————————————————————————————————————
# Third-party scientific stack
import numpy as np                               # array manipulations
import pandas as pd                              # dataframes & CSV I/O
import h5py                                      # HDF5 file I/O
from scipy.optimize import minimize              # classical optimizer
import pylab                                     # MATLAB-like interface

# ————————————————————————————————————————————————
# Plotting
import matplotlib.pyplot as plt                  # plotting library
from qiskit.visualization import (
    plot_histogram,
    plot_bloch_vector
)

# ————————————————————————————————————————————————
# Qiskit Runtime & Providers
from qiskit_ibm_runtime import (
    QiskitRuntimeService,  # authenticate & access IBMQ runtime
    Session,
    EstimatorV2 as RuntimeEstimator
)
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2

# ————————————————————————————————————————————————
# Qiskit Core
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import (
    UnitaryGate,
    EfficientSU2,
    TwoLocal,
    EvolvedOperatorAnsatz,
    CXGate,
    RXXGate, RYYGate, RZZGate, RXGate
)
from qiskit.quantum_info import (
    SparsePauliOp,       # represent Pauli-sum Hamiltonians
    Statevector,
    Operator,
    DensityMatrix,
    entropy,
    random_density_matrix
)

# ————————————————————————————————————————————————
# Qiskit Transpiler & Synthesis
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.synthesis import (
    OneQubitEulerDecomposer,
    TwoQubitBasisDecomposer
)

# ————————————————————————————————————————————————
# Qiskit Algorithms
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import (
    COBYLA, L_BFGS_B, SLSQP, ADAM, GradientDescent, SPSA
)
from qiskit_algorithms.utils import algorithm_globals

# ————————————————————————————————————————————————
# Qiskit Primitives & Simulators
from qiskit.primitives import StatevectorEstimator
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_aer import AerSimulator

# ————————————————————————————————————————————————
# Qiskit Nature (for chemistry/molecular Hamiltonians, if used)
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.mappers import ParityMapper

# ————————————————————————————————————————————————
# Notes:
# - Ensure your IBMQ API token is saved in ~/.qiskit folder or set as an environment variable so that
#   QiskitRuntimeService() can authenticate without additional arguments.
# - You can select between FakeAlmadenV2 (for testing) or real backends via `service.backends()`.
# - Adjust optimizer choice and VQE settings in your VQE run (e.g., maxiter, tol).
# - If you only need statevector simulation, AerSimulator with StatevectorEstimator is sufficient;
#   for runtime execution use RuntimeEstimator with Session context.




# Cleanup: Remove any existing HDF5 gate files before generating new ones
# Pattern matching all previous gate files: "gates_N<...>_D<...>.h5

"""Run this cleanup block at the very start 
of your script or notebook cell—before you call 
save_all_depths!—to ensure no stale HDF5 files remain."""


for filepath in glob.glob(pattern):
    try:
        os.remove(filepath)                                   # delete the file
        print(f"Removed {filepath}")
    except OSError as e:
        print(f"Error removing {filepath}: {e}")              # report any error


### Loading a singular circuit of a chosen depth for fixed N

In [ ]:
"""
load_and_build_circuit.py

This module provides utilities to:
1. Load a saved entangling circuit from an HDF5 file (where each dataset is named "layerX_gateY"),
2. Reconstruct the Qiskit `QuantumCircuit` by appending the corresponding `UnitaryGate`s in the correct order.

Functions:
- `load_gates_hdf5(filename)`: Reads gate matrices from the HDF5 file into a dict keyed by dataset name.
- `generate_quantum_circuit_from_dict(gates_dict)`: Builds and returns a Qiskit `QuantumCircuit` from the loaded gate matrices.
"""

import h5py
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate

def load_gates_hdf5(filename):
    """
    Load gate matrices from an HDF5 file where each dataset is named "layerX_gateY".

    Parameters
    ----------
    filename : str
        Path to the HDF5 file containing the saved gates.

    Returns
    -------
    gates : dict
        A mapping from dataset key ("layerX_gateY") to its NumPy array.
        If the matrix is 4×4 (two-qubit gate) and not the first gate in a layer,
        the matrix is transposed to correct ordering.
    """
    gates = {}
    with h5py.File(filename, "r") as file:
        for key in file.keys():
            # Load the raw matrix data from the dataset
            matrix = file[key][:]
            
            # Parse out the gate index from the key, e.g., "gate2" → 2
            parts = key.split('_')          # ["layerX", "gateY"]
            gate_str = parts[1]             # "gateY"
            gate_index = int(gate_str.replace("gate", ""))

            # If it's a two-qubit gate (4×4 array) and not the first gate,
            # transpose it to match Qiskit's row-major convention if needed.
            if matrix.shape == (4, 4) and gate_index != 1:
                matrix = matrix.T

            gates[key] = matrix
            # Uncomment to debug:
            # print(f"{key} loaded with shape {matrix.shape}")
    return gates

def generate_quantum_circuit_from_dict(gates_dict):
    """
    Build a Qiskit QuantumCircuit from a dict of gate matrices.

    Parameters
    ----------
    gates_dict : dict
        Mapping "layerX_gateY" → NumPy array for that gate.

    Returns
    -------
    qc : QuantumCircuit
        A QuantumCircuit with all layers appended in order. For each layer:
        - Two-qubit gates are applied to qubits (i, i+1) for i=0..n-2.
        - The final gate in the layer (single-qubit) is applied to qubit n-1.
    """
    # Organize matrices into nested dict: layers[L][G] = matrix
    layers = {}
    for key, matrix in gates_dict.items():
        layer_str, gate_str = key.split('_')
        L = int(layer_str.replace("layer", ""))
        G = int(gate_str.replace("gate", ""))
        layers.setdefault(L, {})[G] = matrix

    # Sort the layers and gates within each layer
    sorted_layers = {
        L: [layers[L][g] for g in sorted(layers[L].keys())]
        for L in sorted(layers.keys())
    }

    # Determine number of qubits from gates in the first layer
    first_layer = sorted_layers[next(iter(sorted_layers))]
    n_qubits = len(first_layer)

    # Initialize an empty quantum circuit
    qc = QuantumCircuit(n_qubits)

    # Append each gate in layer-major order
    for L in sorted(sorted_layers.keys()):
        gate_list = sorted_layers[L]
        # Apply two-qubit gates on qubit pairs (0,1), (1,2), ..., (n-2, n-1)
        for i in range(n_qubits - 1):
            mat = gate_list[i]
            gate = UnitaryGate(mat, label=f"L{L}G{i+1}")
            qc.append(gate, [i, i + 1])
        # Apply the final single-qubit gate on qubit (n_qubits - 1)
        mat = gate_list[-1]
        gate = UnitaryGate(mat, label=f"L{L}G{n_qubits}")
        qc.append(gate, [n_qubits - 1])

    return qc

# ————————————————————————————————————————————————
# Example usage:

# 1) Load the matrices from the saved HDF5 file
gates = load_gates_hdf5("gates_N10_D1.h5")

# 2) Reconstruct the QuantumCircuit
qc = generate_quantum_circuit_from_dict(gates)

# 3) Inspect the circuit
print(f"Constructed circuit depth: {qc.depth()}")
qc.draw("mpl")  # visualize with matplotlib

### Loading multiple circuits of a range of depths for fixed N

In [ ]:
import glob
import re
import h5py
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate

def load_gates_hdf5(filename):
    """
    Load gate matrices from an HDF5 file where each dataset is named "layerX_gateY".

    Parameters
    ----------
    filename : str
        Path to the HDF5 file containing saved gate matrices.

    Returns
    -------
    gates : dict[str, np.ndarray]
        Mapping from dataset key ("layerX_gateY") to its NumPy array.
        Two-qubit gates (shape 4×4) except the first in each layer are transposed
        to match Qiskit's convention.
    """
    gates = {}
    with h5py.File(filename, "r") as file:
        for key in file.keys():
            # Read the raw matrix
            matrix = file[key][:]
            # Parse out the gate index from "gateY"
            gate_str = key.split('_')[1]
            gate_idx = int(gate_str.replace("gate", ""))
            # Transpose intermediate 4×4 gates if needed
            if matrix.shape == (4, 4) and gate_idx != 1:
                matrix = matrix.T
            gates[key] = matrix
    return gates

def generate_quantum_circuit_from_dict(gates_dict):
    """
    Build a Qiskit QuantumCircuit from loaded gate matrices.

    Parameters
    ----------
    gates_dict : dict[str, np.ndarray]
        Mapping "layerX_gateY" → matrix array.

    Returns
    -------
    qc : QuantumCircuit
        Circuit that applies each two-qubit gate on (i, i+1) and the final single-qubit gate
        on qubit (n−1), for each layer in ascending order.
    """
    # Group by layer number → { layer_idx: { gate_idx: matrix, ... }, ... }
    layers = {}
    for key, mat in gates_dict.items():
        layer_str, gate_str = key.split('_')
        L = int(layer_str.replace("layer", ""))
        G = int(gate_str.replace("gate", ""))
        layers.setdefault(L, {})[G] = mat

    # Convert to sorted list-of-lists: sorted_layers[0] is gates of layer 1, etc.
    sorted_layers = []
    for L in sorted(layers):
        gate_row = [layers[L][g] for g in sorted(layers[L])]
        sorted_layers.append(gate_row)

    # Determine qubit count from gates in first layer
    n_qubits = len(sorted_layers[0])
    qc = QuantumCircuit(n_qubits)

    # Append gates layer by layer
    for layer_idx, gate_list in enumerate(sorted_layers, start=1):
        # Two-qubit gates on (0,1), (1,2), ..., (n−2,n−1)
        for i, mat in enumerate(gate_list[:-1]):
            gate = UnitaryGate(mat, label=f"L{layer_idx}G{i+1}")
            qc.append(gate, [i, i+1])
        # Single-qubit gate on qubit n−1
        final_mat = gate_list[-1]
        gate = UnitaryGate(final_mat, label=f"L{layer_idx}G{n_qubits}")
        qc.append(gate, [n_qubits-1])

    return qc

def load_all_tnqc_circuits(n_sites, max_depth=10):
    """
    Discover and load tensor-network QC circuits for depths 1..max_depth.

    Scans for files matching "gates_N{n_sites}_D{d}.h5", loads each, reconstructs
    the QuantumCircuit, and returns a dict mapping depth → circuit.

    Parameters
    ----------
    n_sites : int
        Number of qubits/sites in each circuit (encoded in filename).
    max_depth : int, optional
        Maximum depth to load (default is 10).

    Returns
    -------
    circuits : dict[int, QuantumCircuit]
        Mapping from circuit depth d to the loaded QuantumCircuit.
    """
    circuits = {}
    pattern = f"gates_N{n_sites}_D*.h5"
    for fname in sorted(glob.glob(pattern)):
        m = re.match(rf"gates_N{n_sites}_D(\d+)\.h5", fname)
        if not m:
            continue
        depth = int(m.group(1))
        # Only load up to desired max_depth
        if 1 <= depth <= max_depth:
            gates = load_gates_hdf5(fname)
            qc = generate_quantum_circuit_from_dict(gates)
            circuits[depth] = qc
            print(f"Loaded D={depth}: qubits={qc.num_qubits}, depth={qc.depth()}")
    return circuits

# ————————————————————————————————————————————————
# Example invocation:
n_sites = 10
max_depth = 10
circuits_by_depth = load_all_tnqc_circuits(n_sites, max_depth)


### Hamiltonian

In [ ]:
def xyz_hamiltonian(N, Jx, Jy, Jz, h):
    """
    Construct the XYZ spin-½ chain Hamiltonian with a transverse X field as a SparsePauliOp.

    The Hamiltonian is defined (up to a factor) as:
        H = ∑_{j=1}^{N-1} [ Jx·X_j X_{j+1} + Jy·Y_j Y_{j+1} + Jz·Z_j Z_{j+1} ]
            – h · ∑_{j=1}^N X_j

    Here we absorb the usual factor of 1/4 (for spin-½ Pauli matrices) into the coefficients.

    Parameters
    ----------
    N : int
        Number of qubits (sites) in the chain.
    Jx, Jy, Jz : float
        Coupling strengths for the XX, YY, and ZZ interactions, respectively.
    h : float
        Transverse field strength along the X direction.

    Returns
    -------
    SparsePauliOp
        The Hamiltonian expressed as a sum of Pauli terms.
    """
    pauli_labels = []   # List of N-character strings like "XXIII…", etc.
    coeffs = []         # Corresponding list of numeric coefficients

    # 1) Nearest-neighbor exchange interactions
    for j in range(N - 1):
        # -- XX term on sites j, j+1
        label = ['I'] * N
        label[j], label[j + 1] = 'X', 'X'
        pauli_labels.append(''.join(label))
        coeffs.append(Jx / 4)   # factor 1/4 from Pauli matrices

        # -- YY term
        label = ['I'] * N
        label[j], label[j + 1] = 'Y', 'Y'
        pauli_labels.append(''.join(label))
        coeffs.append(Jy / 4)

        # -- ZZ term
        label = ['I'] * N
        label[j], label[j + 1] = 'Z', 'Z'
        pauli_labels.append(''.join(label))
        coeffs.append(Jz / 4)

    # 2) Transverse X field on each site
    for j in range(N):
        label = ['I'] * N
        label[j] = 'X'
        pauli_labels.append(''.join(label))
        coeffs.append(-h / 2)  # factor 1/2 for spin-½

    # Build and return the sparse Pauli operator
    return SparsePauliOp(pauli_labels, coeffs)


# ————————————————————————————————————————————————
# Example parameters and Hamiltonian construction - have to be the same parameter values used to generate the ground state MPS (psi_non_truncated):
N  = 10   # number of qubits/sites
Jx = 5.0  # coupling Sx–Sx between neighbors
Jy = 6.0  # coupling Sy–Sy
Jz = 5.0  # coupling Sz–Sz
h  = 0.5  # transverse field strength

hamiltonian = xyz_hamiltonian(N, Jx, Jy, Jz, h)
print(hamiltonian)  # inspect terms, shapes, etc.


### Sanity Check: Ensure hamiltonian is correct, as well the circuit has been correctly interpreted in your Qiskit environment

In [ ]:
# ————————————————————————————————————————————————
# Verify Hamiltonian & Circuit Initial Guess vs. Exact Ground State

from qiskit.quantum_info import Statevector
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
import numpy as np

# (Assumes `qc` is your Qiskit QuantumCircuit and `hamiltonian` is the SparsePauliOp from above)

# 1) Build the statevector of the ansatz circuit
state = Statevector(qc)

# 2) Compute the expectation value ⟨ψ|H|ψ⟩ for the initial guess
Energy = state.expectation_value(hamiltonian)
print("Theoretical energy of initial guess:", np.real(Energy))

# 3) Solve for the exact ground-state energy via a classical eigensolver
numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
groundstate_value = result.eigenvalue.real
print(f"Ground-state energy (exact): {groundstate_value:.5f}")


# Addition of Variational Tail

### Adding a variational Tail of depth p to a circuit of depth D

In [ ]:
"""
Append p layers of parametrized two-qubit XX/YY/ZZ interactions and single-qubit RX rotations
to an existing fixed circuit `qc`, yielding the full variational ansatz `full_circ`.

Each layer consists of:
  - RXX, RYY, RZZ gates on every neighbor pair (j, j+1),
  - RX rotations on every qubit j.

Parameters are named sequentially "ϕ_0", "ϕ_1", … for use in VQE.
"""

# Number of qubits in the original circuit
n_qubits = qc.num_qubits

# Variational depth: number of repeated layers to add
p = 2

# Will hold all Parameter objects in order
params = []

# Build an empty QuantumCircuit for the variational ansatz
ansatz = QuantumCircuit(n_qubits)

# --- Construct p variational layers ---
for layer in range(p):
    # Two-qubit entangling gates on each neighbor pair
    for j in range(n_qubits - 1):
        # RXX parameterized gate
        phi = Parameter(f"ϕ_{len(params)}")
        params.append(phi)
        ansatz.append(RXXGate(phi), [j, j+1])

        # RYY parameterized gate
        phi = Parameter(f"ϕ_{len(params)}")
        params.append(phi)
        ansatz.append(RYYGate(phi), [j, j+1])

        # RZZ parameterized gate
        phi = Parameter(f"ϕ_{len(params)}")
        params.append(phi)
        ansatz.append(RZZGate(phi), [j, j+1])

    # Single-qubit RX rotations on each qubit
    for j in range(n_qubits):
        phi = Parameter(f"ϕ_{len(params)}")
        params.append(phi)
        ansatz.append(RXGate(phi), [j])

# --- Combine the fixed depth-D circuit with the variational ansatz ---
# `compose` appends `ansatz` to the end of `qc`
full_circ = qc.compose(ansatz)

# Visualize the resulting circuit
full_circ.draw("mpl")


### Adding a range of variational Tail of depths p to a range of circuit with varying depths D

In [ ]:
def generate_augmented_circuits(circuits_by_depth, max_p=8):
    """
    Build a family of “hybrid” circuits by appending p layers of variational gates
    onto each base MPS-derived circuit of depth D.

    Each appended “tail” layer contains:
      - RXX, RYY, RZZ rotations on every neighboring qubit pair,
      - RX rotations on every single qubit.

    Parameters
    ----------
    circuits_by_depth : dict[int, QuantumCircuit]
        Mapping from MPS circuit depth D to the fixed QuantumCircuit (base_qc).
    max_p : int, optional
        Maximum number of variational layers to append (default 8).

    Returns
    -------
    augmented : dict[tuple[int,int], QuantumCircuit]
        Dictionary keyed by (D, p), where D is the base circuit depth and
        p the number of appended variational layers. Each value is the
        composed QuantumCircuit named `"MPS{D}_var{p}"`.
    """
    augmented = {}

    # Loop over each existing MPS circuit depth D
    for D, base_qc in circuits_by_depth.items():
        n_qubits = base_qc.num_qubits  # number of qubits in this circuit

        # For each variational depth p = 1..max_p
        for p in range(1, max_p + 1):
            params = []  # will collect all Parameter objects for (D, p)
            # Build an empty “tail” circuit to append
            tail = QuantumCircuit(n_qubits, name=f"var_D{D}_p{p}")

            # Add p variational layers
            for layer_idx in range(p):
                # 1) Two-qubit parameterized rotations on each neighbor pair
                for q in range(n_qubits - 1):
                    for GateClass in (RXXGate, RYYGate, RZZGate):
                        φ = Parameter(f"ϕ_D{D}_p{p}_{len(params)}")
                        params.append(φ)
                        tail.append(GateClass(φ), [q, q + 1])

                # 2) Single-qubit RX rotations on every qubit
                for q in range(n_qubits):
                    φ = Parameter(f"ϕ_D{D}_p{p}_{len(params)}")
                    params.append(φ)
                    tail.append(RXGate(φ), [q])

            # Compose the MPS-derived core circuit with the variational tail
            full = base_qc.compose(tail, inplace=False)
            full.name = f"MPS{D}_var{p}"  # assign a descriptive name
            augmented[(D, p)] = full

    return augmented

# ————————————————————————————————————————————————
# Example invocation:
augmented_circuits = generate_augmented_circuits(circuits_by_depth, max_p=8)

# Inspect one hybrid circuit, e.g., base MPS depth D=4 with p=2 variational layers
# full_circ = augmented_circuits[(4, 2)]
# full_circ.draw("mpl")


# VQE Optimisation

### Single Circuit Optimisation: Statevector 

In [ ]:
"""
Assumes:
- `full_circ` is your Qiskit QuantumCircuit with Parameters `params`.
- `hamiltonian` is defined (either a NumPy array or a Qiskit Operator/SparsePauliOp).
- `groundstate_value` holds the exact ground-state energy (for comparison).
- Matplotlib (`plt`) is imported.
"""

In [ ]:
"""
vqe_statevector.py

Perform a VQE-style optimization of the variational circuit `full_circ` 
against the XYZ Hamiltonian using a classical statevector-based cost function
and SciPy’s `minimize`. Records energy at each iteration and plots convergence.
"""

import numpy as np
from qiskit.quantum_info import Statevector, Operator
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def energy(param_vals):
    """
    Compute ⟨ψ(θ)| H |ψ(θ)⟩ for the current parameter vector `param_vals`.

    1. Bind `param_vals` to `full_circ` to get a concrete circuit.
    2. Simulate its statevector |ψ⟩.
    3. Convert `hamiltonian` to a dense matrix if needed.
    4. Compute the Rayleigh quotient: real(ψ† H ψ).

    Parameters
    ----------
    param_vals : array_like
        Values for each Parameter in `params`, in the same order.

    Returns
    -------
    float
        The real expectation value of H under |ψ(param_vals)⟩.
    """
    # 1) Bind parameters to the circuit (does not modify `full_circ` in place)
    bound_qc = full_circ.assign_parameters(
        {p: v for p, v in zip(params, param_vals)},
        inplace=False
    )
    # 2) Get the statevector as a complex NumPy array
    psi = Statevector.from_instruction(bound_qc).data

    # 3) Ensure we have a dense Hamiltonian matrix
    if isinstance(hamiltonian, np.ndarray):
        H_mat = hamiltonian
    else:
        H_mat = Operator(hamiltonian).data

    # 4) Compute ⟨ψ|H|ψ⟩ = ψ† (H ψ), take the real part
    return np.real(np.vdot(psi, H_mat.dot(psi)))

# --- Initialize parameters and storage ---
init_vals = np.ones(len(params))   # Starting guess (all ones)
energies = []                      # To record energy at each callback

def record_energy(xk):
    """
    Callback for the optimizer: record the current energy value.
    SciPy passes the current parameter vector `xk` here.
    """
    energies.append(energy(xk))

# --- Run the classical optimization (e.g., Conjugate Gradient) ---
res = minimize(
    energy,           # objective function
    init_vals,        # initial parameter vector
    method='CG',      # Conjugate Gradient
    callback=record_energy
)
opt_vals = res.x   # Optimized parameter values

# --- Print summary comparison ---
if 'groundstate_value' in globals():
    print("Energy of TNQC (initial MPS circuit):", np.real(Energy))
    print("Energy of initial variational guess:", np.real(energies[0]))
    print(f"Optimized variational energy: {res.fun:.8f}")
    print(f"Exact ground-state energy: {groundstate_value:.8f}")
    print(f"Absolute error: {res.fun - groundstate_value:.2e}")
    print(f"Percentage error: {abs((res.fun - groundstate_value) / groundstate_value) * 100:.2f}%")

# --- Plot convergence ---
plt.figure()
plt.plot(energies, marker='o')
plt.xlabel('Iteration')
plt.ylabel('Energy ⟨ψ|H|ψ⟩')
plt.title('Variational Energy vs. Iteration')
plt.grid(True)
plt.tight_layout()
plt.show()


### Single Circuit Optimisation: Shot and Seed Number

In [ ]:
"""
shot_vqe.py

Perform shot-based VQE optimization using AerEstimator and SciPy COBYLA.
Parameters are updated via a classical optimizer; energies at each iteration
are estimated with finite shots and recorded for convergence analysis.
"""

import numpy as np
import matplotlib.pyplot as plt
from qiskit.primitives import Estimator as AerEstimator
from qiskit.quantum_info import Operator
from scipy.optimize import minimize

# ————————————————————————————————————————————————
# 1) Configure shot-based estimator for reproducibility
seed   = 75
shots  = 1024
estimator = AerEstimator(
    run_options={"seed": seed, "shots": shots},        # RNG seed + shot count
    transpile_options={"seed_transpiler": seed}       # transpiler randomness seed
)

# 2) Ensure our Hamiltonian is a Qiskit Operator (not a raw NumPy array)
H_op = hamiltonian if not isinstance(hamiltonian, np.ndarray) else Operator(hamiltonian)

# ————————————————————————————————————————————————
# 3) Define the cost function using finite-shot estimation
def energy(param_vals):
    """
    Bind `param_vals` to the full circuit, run it on AerEstimator,
    and return the estimated expectation value ⟨H⟩.
    """
    # Bind parameters (does not mutate `full_circ`)
    bound_qc = full_circ.assign_parameters(
        {p: v for p, v in zip(params, param_vals)},
        inplace=False
    )
    # Execute and fetch results
    job    = estimator.run([bound_qc], [H_op])
    result = job.result()
    # Return the real part of the first (and only) estimated value
    return float(np.real(result.values[0]))

# 4) Initialize optimizer inputs
init_vals = np.ones(len(params))  # starting guess for all angles
energies  = []                    # to record ⟨H⟩ over iterations

def record_energy(xk):
    """Callback for SciPy minimize to store the current energy."""
    energies.append(energy(xk))

# 5) Run the classical optimizer (COBYLA) with a 2000-iteration limit
res      = minimize(
    energy,
    init_vals,
    method='COBYLA',
    callback=record_energy,
    options={'maxiter': 2000}
)
opt_vals = res.x

# 6) Print comparison against exact ground state (if available)
if 'groundstate_value' in globals():
    print("Energy of TNQC:", np.real(Energy))
    print("Energy of initial guess:", np.real(energies[0]))
    print(f"Optimized energy: {res.fun:.8f}")
    print(f"Exact ground-state energy: {groundstate_value:.8f}")
    print(f"Absolute error: {res.fun - groundstate_value:.2e}")
    print(f"Percentage error: {abs((res.fun - groundstate_value) / groundstate_value) * 100:.2f}%")

# 7) Plot VQE energy convergence
plt.figure()
plt.plot(energies, marker='o')
plt.xlabel('Iteration')
plt.ylabel('Energy ⟨ψ|H|ψ⟩')
plt.title('Shot-based VQE Energy Convergence')
plt.grid(True)
plt.tight_layout()
plt.show()


### Multiple Circuit Optimisation: Statevector

In [ ]:
# ————————————————————————————————————————————————
# Batch VQE Optimization over Hybrid Circuits (Statevector-based)

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from qiskit.quantum_info import Statevector, Operator

def optimize_all(augmented_circuits, hamiltonian, method='COBYLA'):
    """
    Perform statevector-based VQE optimization for each (D, p) hybrid circuit.

    Parameters
    ----------
    augmented_circuits : dict[tuple[int,int], QuantumCircuit]
        Mapping (MPS_depth D, var_depth p) → corresponding QuantumCircuit.
    hamiltonian : SparsePauliOp or np.ndarray
        The problem Hamiltonian.
    method : str
        Name of the SciPy optimization method (default 'COBYLA').

    Returns
    -------
    results : dict[tuple[int,int], dict]
        For each (D, p), returns a dict with:
        - 'res': the OptimizeResult from SciPy,
        - 'energies': list of energy values recorded each callback,
        - 'percentage error': final percent error w.r.t. `groundstate_value`.
    """
    results = {}

    # Loop over every circuit with its MPS and variational depths
    for (D, p), qc in augmented_circuits.items():
        # Extract parameters in a reproducible, sorted order
        params = list(qc.parameters)
        n_params = len(params)

        # Define the cost function ⟨ψ|H|ψ⟩ via statevector
        def energy(param_vals):
            # 1) Bind parameters to the circuit
            bound_qc = qc.assign_parameters(
                {param: val for param, val in zip(params, param_vals)},
                inplace=False
            )
            # 2) Simulate to get statevector |ψ⟩
            psi = Statevector.from_instruction(bound_qc).data
            # 3) Convert Hamiltonian to dense matrix if needed
            H_mat = (hamiltonian if isinstance(hamiltonian, np.ndarray)
                     else Operator(hamiltonian).data)
            # 4) Compute Rayleigh quotient ⟨ψ|H|ψ⟩
            return np.real(np.vdot(psi, H_mat.dot(psi)))

        # Prepare to record energy at each optimization step
        energies = []
        def callback(xk):
            energies.append(energy(xk))

        # Initial guess: all ones
        init = np.ones(n_params)

        # Run the optimizer
        res = minimize(energy, init, method=method, callback=callback)

        # Compute percent error against known exact ground state
        pct_err = abs((res.fun - groundstate_value) / groundstate_value) * 100

        # Store results for this (D, p)
        results[(D, p)] = {
            'res': res,
            'energies': energies,
            'percentage error': pct_err
        }

    return results

# Run the batch optimization
all_results = optimize_all(augmented_circuits, hamiltonian, method='COBYLA')


### Multiple Circuit Optimisation: Shot and Seed Number

In [ ]:
"""
batch_shot_vqe.py

Run shot-based VQE across multiple hybrid circuits (MPS depth D + variational tail p),
using AerEstimator with fixed seed and shot count. Records percentage error for each (D, p).
"""

import numpy as np
from scipy.optimize import minimize
from qiskit.quantum_info import Operator
from qiskit.primitives import Estimator as AerEstimator

# 1) Configure the shot-based estimator for reproducibility
seed  = 75
shots = 1024
estimator = AerEstimator(
    run_options={"seed": seed, "shots": shots},
    transpile_options={"seed_transpiler": seed},
)

# 2) Ensure Hamiltonian is in the correct format (SparsePauliOp or Qiskit Operator)
H_op = Operator(hamiltonian) if isinstance(hamiltonian, np.ndarray) else hamiltonian

# 3) Prepare to collect results for each circuit
records = []

# 4) Loop over each hybrid circuit of depth (D, p)
for (D, p), qc in augmented_circuits.items():
    n_qubits = qc.num_qubits

    # 4a) Identify the parameter list specific to this (D, p) ansatz
    params = [
        param
        for param in qc.parameters
        if param.name.startswith(f"ϕ_D{D}_p{p}_")
    ]

    # 4b) Define the shot-based energy estimation function
    def energy_fn(x, qc=qc, params=params, H=H_op):
        """
        Bind parameters x to qc, run AerEstimator, and return ⟨H⟩ estimate.
        """
        # Bind parameter values to the circuit (non-destructive)
        bound = qc.assign_parameters(
            {p: v for p, v in zip(params, x)},
            inplace=False
        )
        # Execute the circuit and measure H
        job    = estimator.run(circuits=[bound], observables=[H])
        result = job.result()
        return float(np.real(result.values[0]))

    # 4c) Initialize all parameters to 1.0
    init_vals = np.ones(len(params))

    # 4d) Run classical optimizer (COBYLA with up to 2000 iterations)
    res = minimize(
        energy_fn,
        init_vals,
        method='COBYLA',
        options={'maxiter': 2000}
    )

    # 4e) Compute percentage error relative to exact ground state
    pct_err = abs((res.fun - groundstate_value) / groundstate_value) * 100

    # 4f) Store results for this circuit
    records.append({
        'qubits':    n_qubits,
        'MPS_depth': D,
        'var_depth': p,
        'pct_error': pct_err
    })

### Multiple Circuit Optimisation: Best Circuit Composition and Performance Landscape

In [ ]:
# ————————————————————————————————————————————————
# Compile Results into DataFrame and Visualize Best Circuit & Error Landscape

import matplotlib.pyplot as plt

# 1) Flatten the results dict into a list of records
records = []
for (D, p), data in all_results.items():
    records.append({
        'MPS_depth':   D,
        'var_depth':   p,
        'pct_error':   data['percentage error'],
        'final_energy': data['res'].fun
    })

# 2) Build and sort the DataFrame
df = pd.DataFrame(records)
df = df.sort_values(['MPS_depth', 'var_depth']).reset_index(drop=True)
print(df.to_string(index=False))

# 3) Identify the circuit with minimum percentage error
best_idx = df['pct_error'].idxmin()
best_row = df.loc[best_idx]
D_best   = int(best_row['MPS_depth'])
p_best   = int(best_row['var_depth'])
err_best = best_row['pct_error']
print(f"Lowest error = {err_best:.4f}% at MPS_depth={D_best}, var_depth={p_best}")

# Retrieve and display that optimal circuit
best_circ = augmented_circuits[(D_best, p_best)]
best_circ.draw('mpl')

# 4) Visualize error landscape as a 3D surface
pivot = df.pivot(index='MPS_depth', columns='var_depth', values='pct_error')

X, Y = np.meshgrid(pivot.columns.values, pivot.index.values)
Z = pivot.values

fig = plt.figure(figsize=(9, 6))
ax  = fig.add_subplot(111, projection='3d')

# Surface plot of percentage error vs (D, p)
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

# Label axes
ax.set_xlabel('Variational Tail Depth (p)', labelpad=10)
ax.set_ylabel('TNQC Depth (D)',        labelpad=10)
ax.set_zlabel('Percentage Error (%)',  labelpad=15)
ax.set_title('Error Landscape vs TNQC Depth and Variational Depth', fontsize=13, y=1.0)

# Colorbar
cbar = fig.colorbar(surf, ax=ax, shrink=0.6, aspect=12, pad=0.12)
cbar.ax.set_ylabel('Percentage Error (%)', rotation=90, labelpad=10)

# Adjust layout and viewing angle
fig.subplots_adjust(left=0.10, right=0.85, bottom=0.10, top=0.90)
ax.view_init(elev=25, azim=170)

plt.show()
